In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
# import os
from sklearn.linear_model import LinearRegression
import sklearn as skl
from sklearn import metrics
from sklearn.model_selection import train_test_split
# import statsmodels.api as sm
# from scipy import stats
# pd.set_option('max_columns', None)
# pd.set_option('max_rows', None)

In [2]:
train_df = pd.read_csv('./datasets/train.csv')

In [3]:
train_df.head(1)

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500


In [4]:
# Looking for missing values
train_df.isnull().sum()

Id                0
PID               0
MS SubClass       0
MS Zoning         0
Lot Frontage    330
               ... 
Misc Val          0
Mo Sold           0
Yr Sold           0
Sale Type         0
SalePrice         0
Length: 81, dtype: int64

In [5]:
#function to return unique values for columns, not including columns with more than 100 unique values (ie discrete and continuous)
def columns_values(dataframe):
    for n in dataframe.columns:
        column = n
    #     print(column)
        value_counts = (dataframe[column].value_counts())
        unique_values = (dataframe[column].unique())
        len_counts = len(value_counts)
        len_unique = len(unique_values)
        if (len(value_counts)< 100):
#             if 'np.nan' in unique_values:
#             if len_counts != len(value_counts):
                print(f"{column} has length of {len_unique}")
#                 print(len(unique_values))
                print(unique_values)
                print("="*30)
                print(f"{column} has length of {len_counts}")
#                 print(len(value_counts))
                print(value_counts)
                print("="*60)

In [6]:
#running the function to view unique values
columns_values(train_df)

MS SubClass has length of 16
[ 60  20  50 180 160  70 120 190  85  30  90  80  75  45  40 150]
MS SubClass has length of 16
20     770
60     394
50     198
120    132
30     101
70      90
160     88
80      86
90      75
190     46
85      28
75      16
45      11
180     11
40       4
150      1
Name: MS SubClass, dtype: int64
MS Zoning has length of 7
['RL' 'RM' 'FV' 'C (all)' 'A (agr)' 'RH' 'I (all)']
MS Zoning has length of 7
RL         1598
RM          316
FV          101
C (all)      19
RH           14
A (agr)       2
I (all)       1
Name: MS Zoning, dtype: int64
Street has length of 2
['Pave' 'Grvl']
Street has length of 2
Pave    2044
Grvl       7
Name: Street, dtype: int64
Alley has length of 3
[nan 'Pave' 'Grvl']
Alley has length of 2
Grvl    85
Pave    55
Name: Alley, dtype: int64
Lot Shape has length of 4
['IR1' 'Reg' 'IR2' 'IR3']
Lot Shape has length of 4
Reg    1295
IR1     692
IR2      55
IR3       9
Name: Lot Shape, dtype: int64
Land Contour has length of 4
['Lvl' 'HL

0.0    1200
1.0     824
2.0      23
3.0       2
Name: Bsmt Full Bath, dtype: int64
Bsmt Half Bath has length of 4
[ 0.  1. nan  2.]
Bsmt Half Bath has length of 3
0.0    1923
1.0     122
2.0       4
Name: Bsmt Half Bath, dtype: int64
Full Bath has length of 5
[2 1 3 0 4]
Full Bath has length of 5
2    1096
1     900
3      45
0       8
4       2
Name: Full Bath, dtype: int64
Half Bath has length of 3
[1 0 2]
Half Bath has length of 3
0    1308
1     725
2      18
Name: Half Bath, dtype: int64
Bedroom AbvGr has length of 8
[3 4 2 5 1 0 6 8]
Bedroom AbvGr has length of 8
3    1108
2     544
4     265
1      75
5      41
6      12
0       5
8       1
Name: Bedroom AbvGr, dtype: int64
Kitchen AbvGr has length of 4
[1 2 0 3]
Kitchen AbvGr has length of 4
1    1960
2      88
0       2
3       1
Name: Kitchen AbvGr, dtype: int64
Kitchen Qual has length of 4
['Gd' 'TA' 'Fa' 'Ex']
Kitchen Qual has length of 4
TA    1047
Gd     806
Ex     151
Fa      47
Name: Kitchen Qual, dtype: int64
TotRms Ab

In [7]:
#building dictionaries to replace values
replace_values = {'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1, 
     'AllPub': 4, 'NoSewr': 3, 'NoSeWa': 2, 'ELO': 1,
     'Gtl': 3, 'Mod': 2, 'Sev': 1,
     'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1,
     'Gd': 4, 'Av': 3, 'Mn': 2, 'No':1,
     'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1,
     'N': 0, 'Y': 1,
     'SBrkr': 5, 'FuseA': 4, 'FuseF': 3, 'FuseP': 2, 'Mix': 1,
     'Typ': 8, 'Min1': 7, 'Min2': 6, 'Mod': 5, 'Maj1': 4, 'Maj2': 3, 'Sev': 2, 'Sal': 1,
     'Fin': 3, 'RFn': 2, 'Unf': 1,
     'Y': 3, 'P': 2, 'N': 1,
     'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'NA' : 0
     }
# train_df = train_df.replace(replace_values)
replace_values_na = {np.nan: "NA"}
replace_values_nan = {np.nan: 0}

In [8]:
# using the dictionaries to replace values
train_df['Bsmt Qual'] = train_df['Bsmt Qual'].replace(replace_values_nan)
train_df['Bsmt Cond'] = train_df['Bsmt Cond'].replace(replace_values_nan)
train_df['Bsmt Exposure'] = train_df['Bsmt Exposure'].replace(replace_values_nan)
train_df['BsmtFin Type 1'] = train_df['BsmtFin Type 1'].replace(replace_values_nan)
train_df['BsmtFin Type 2'] = train_df['BsmtFin Type 2'].replace(replace_values_nan)
train_df['Bsmt Full Bath'] = train_df['Bsmt Full Bath'].replace(replace_values_nan)
train_df['Bsmt Half Bath'] = train_df['Bsmt Half Bath'].replace(replace_values_nan)
train_df['Fireplace Qu'] = train_df['Fireplace Qu'].replace(replace_values_nan)
train_df['Garage Finish'] = train_df['Garage Finish'].replace(replace_values_nan)
train_df['Garage Qual'] = train_df['Garage Qual'].replace(replace_values_nan)
train_df['Garage Cond'] = train_df['Garage Cond'].replace(replace_values_nan)
train_df['Fireplace Qu'] = train_df['Fireplace Qu'].replace(replace_values_nan)
train_df['Garage Cars'] = train_df['Garage Cars'].replace(replace_values_nan)
train_df['Garage Area'] = train_df['Garage Area'].replace(replace_values_nan)
train_df['Pool QC'] = train_df['Pool QC'].replace(replace_values_nan)
train_df['Fence'] = train_df['Fence'].replace(replace_values_nan)
train_df['Lot Frontage'] = train_df['Lot Frontage'].replace(replace_values_nan)
train_df['Alley'] = train_df['Alley'].replace(replace_values_na)
train_df['Mas Vnr Type'] = train_df['Mas Vnr Type'].replace(replace_values_na)
train_df['Mas Vnr Area'] = train_df['Mas Vnr Area'].replace(replace_values_nan)
train_df['Mas Vnr Area'] = train_df['Mas Vnr Area'].replace(replace_values_nan)
train_df['BsmtFin SF 1'] = train_df['BsmtFin SF 1'].replace(replace_values_nan)
train_df['BsmtFin SF 2'] = train_df['BsmtFin SF 2'].replace(replace_values_nan)
train_df['Bsmt Unf SF'] = train_df['Bsmt Unf SF'].replace(replace_values_nan)
train_df['Total Bsmt SF'] = train_df['Total Bsmt SF'].replace(replace_values_nan)
train_df['Garage Type'] = train_df['Garage Type'].replace(replace_values_na)
train_df['Garage Yr Blt'] = train_df['Garage Yr Blt'].replace({np.nan: 1980})
train_df['Misc Feature'] = train_df['Misc Feature'].replace(replace_values_na)
train_df = train_df.replace(replace_values)

In [ ]:
train_df.columns = train_df.columns.str.lower().str.replace(' ', '_')

In [10]:
#running function to review unique values again
columns_values(train_df)

ms_subclass has length of 16
[ 60  20  50 180 160  70 120 190  85  30  90  80  75  45  40 150]
ms_subclass has length of 16
20     770
60     394
50     198
120    132
30     101
70      90
160     88
80      86
90      75
190     46
85      28
75      16
45      11
180     11
40       4
150      1
Name: ms_subclass, dtype: int64
ms_zoning has length of 7
['RL' 'RM' 'FV' 'C (all)' 'A (agr)' 'RH' 'I (all)']
ms_zoning has length of 7
RL         1598
RM          316
FV          101
C (all)      19
RH           14
A (agr)       2
I (all)       1
Name: ms_zoning, dtype: int64
street has length of 2
['Pave' 'Grvl']
street has length of 2
Pave    2044
Grvl       7
Name: street, dtype: int64
alley has length of 3
[0 'Pave' 'Grvl']
alley has length of 3
0       1911
Grvl      85
Pave      55
Name: alley, dtype: int64
lot_shape has length of 4
[3 4 2 1]
lot_shape has length of 4
4    1295
3     692
2      55
1       9
Name: lot_shape, dtype: int64
land_contour has length of 4
['Lvl' 'HLS' 'Bnk' 

mo_sold has length of 12
[ 3  4  1  6  5  9  7  2 12 10 11  8]
mo_sold has length of 12
6     352
7     303
5     257
4     208
3     168
8     167
10    123
9     109
2     104
11    103
1      79
12     78
Name: mo_sold, dtype: int64
yr_sold has length of 5
[2010 2009 2006 2007 2008]
yr_sold has length of 5
2007    498
2009    446
2006    438
2008    435
2010    234
Name: yr_sold, dtype: int64
sale_type has length of 9
['WD ' 'New' 'COD' 'ConLD' 'Con' 'CWD' 'Oth' 'ConLI' 'ConLw']
sale_type has length of 9
WD       1781
New       160
COD        63
ConLD      17
CWD        10
ConLI       7
ConLw       5
Con         4
Oth         4
Name: sale_type, dtype: int64


In [11]:
# checking to make sure that missing values are corrected
train_df.isnull().sum()

id              0
pid             0
ms_subclass     0
ms_zoning       0
lot_frontage    0
               ..
misc_val        0
mo_sold         0
yr_sold         0
sale_type       0
saleprice       0
Length: 81, dtype: int64

In [12]:
train_df.head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,0.0,13517,Pave,0,3,Lvl,...,0,0,0,0,0,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,0,3,Lvl,...,0,0,0,0,0,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,0,4,Lvl,...,0,0,0,0,0,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,0,4,Lvl,...,0,0,0,0,0,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,0,3,Lvl,...,0,0,0,0,0,0,3,2010,WD,138500


In [23]:
for column in train_df:
    print(type(train_df[column]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.S

In [13]:
from sklearn.preprocessing import PolynomialFeatures


In [15]:
X = train_df.drop('saleprice', axis=1)
y = train_df['saleprice']

poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_overfit = poly.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_overfit,
    y,
    random_state=42)
sc = StandardScaler()
Z_train = sc.fit_transform(X_train)
Z_test = sc.transform(X_test)

ValueError: could not convert string to float: 'RL'

In [ ]:
# setting up my correlation check for saleprice
corr = train_df.corr()
saleprice_corr = corr['SalePrice']
saleprice_corr > .5

In [ ]:
drop_features = []
features = []
for index, column in saleprice_corr.items():
    if column < .5:
        drop_features.append(index)
    else:
        features.append(index)

In [ ]:
for column in train_df.columns:
    if column in drop_features:
        train_df.drop([column], axis=1, inplace=True)

In [ ]:
train_df.dtypes

In [ ]:
for column in train_df:
    print(column)

In [ ]:
def create_plots(dataframe, column1):
    for column in dataframe:
        try:
            fig, ax = plt.subplots(figsize=(10,6))
            plt.scatter(train_df[column1], train_df[column])
            plt.title(column, fontsize=25)
            plt.xlabel(column1)
            plt.ylabel(column)
            plt.show();
        except:
            continue

In [ ]:
def create_plot(dataframe, column1, column2):
    fig, ax = plt.subplots(figsize=(10,6))
    plt.scatter(train_df[column1], train_df[column2])
    plt.title(column2, fontsize=25)
    plt.xlabel(column1)
    plt.ylabel(column2)
    plt.show();

In [24]:
train_df.dtypes

id                int64
pid               int64
ms_subclass       int64
ms_zoning        object
lot_frontage    float64
                 ...   
misc_val          int64
mo_sold           int64
yr_sold           int64
sale_type        object
saleprice         int64
Length: 81, dtype: object

In [ ]:
# create_plots(train_df, 'SalePrice')

In [ ]:
train_df = train_df[(train_df['Gr Liv Area'] < 4000)]

In [ ]:
train_df.corr()['SalePrice']

In [ ]:
train_df['Exter Qual and Gr Liv Area'] = train_df['Exter Qual'] * train_df['Gr Liv Area']

In [ ]:
train_df.corr()['SalePrice']

In [ ]:
train_df.shape

In [ ]:
features = ['Overall Qual', 'Year Built', 'Mas Vnr Area', 'Exter Qual', 'Total Bsmt SF', '1st Flr SF', 'Gr Liv Area', 'TotRms AbvGrd', 'Fireplace Qu', 'Garage Area', 'Exter Qual and Gr Liv Area']
X = train_df[features]
y = train_df['SalePrice']

In [ ]:
len(features)

In [ ]:
len(X)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,)
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.intercept_
lr.coef_
y_preds = lr.predict(X)
resids = y - y_preds
print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

In [ ]:
def return_reg_metrics(X, y, k):
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    lr.coef_
    lr.intercept_
    preds = lr.predict(X)
    resids = y - preds
    sse = (resids**2).sum()
    null_pred = y.mean()
    null_resids = y - null_pred
    null_sse = (null_resids**2).sum()  
    print("The SSE = " + str((resids**2).sum()))
    print("The MSE = " + str(metrics.mean_squared_error(y, preds)))
    print("The RMSE = " + str((metrics.mean_squared_error(y, preds, squared=False))))
    print("The R2 = " + str((metrics.r2_score(y, preds))))
    n = len(X)
    r2 = 1 - (sse/null_sse)
    r2adj_top = ((1 - r2)*(n - 1))
    r2adj_bot = (n - k - 1)
    r2adj = 1- (r2adj_top / r2adj_bot)
    print("The R2ADJ = " + str((r2adj)))

In [ ]:
return_reg_metrics(X, y, 3)

In [ ]:
'Garage Cars', 'Mas Vnr Area', 'Bsmt Qual', '1st Flr SF', 'Full Bath', 'Kitchen Qual', 'Fireplace Qu', 'Garage Finish', 'Garage Cars',

In [ ]:
lr.coef_